In [ ]:
# from robust_pricing.path_generators import BinomialTree, Uniform, Gaussian, GaussianMartingale, UniformMartingale
from martingale_optimal_transport.discrete import *
from robust_pricing.marginals_creators.binomial_generator import BinomialGenerator

from gurobipy import GRB
import torch
import numpy as np

In [ ]:
max_steps = 100

volatility = 0.19
granularity = max_steps

up_factor = np.exp(volatility * np.sqrt(1 / granularity))
down_factor = 1 / up_factor

mu = BinomialGenerator(100, up=up_factor, time_horizon=granularity)

In [ ]:
up_factor

In [ ]:
path_length = 4
observed_times = [i * (granularity // path_length) for i in range(1, path_length + 1)]

mu_0 = [mu.marginals[i] for i in observed_times]

In [ ]:
# K = 100
# def f(x):
#     if isinstance(x, torch.Tensor):
#         return torch.relu(x.select(1,-1) - K)
#     else:
#         return max(x[-1] - K, 0)
    
    
def f(x):
    if isinstance(x, torch.Tensor):
        return torch.relu(x.select(1,-1) - torch.mean(x, 1))
    else:
        return max(x[-1] - sum(x)/len(x), 0)

In [ ]:
mot_sup = MartingaleOptimalTransport(
    mu=mu_0, 
    cost_function=f,
    sense=GRB.MAXIMIZE
)

mot_inf = MartingaleOptimalTransport(
    mu=mu_0, 
    cost_function=f,
    sense=GRB.MINIMIZE
)

In [ ]:
mot_sup.solve()

In [ ]:
mot_inf.solve()

In [ ]:
mot_sup.solution_object.draw_slice(1)
mot_inf.solution_object.draw_slice(1)

In [ ]:
print(mot_sup.model.obj_val, mot_inf.model.obj_val)

In [ ]:
# n =100 -> 6.768039845540382 5.694280587662192
# n =1000 -> 6.772804520574041 5.643967968517473